In [9]:
pip install transformers torch torchvision matplotlib


Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from PIL import Image
import torch
import requests

# Load processor and model
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# Test on an image
image = Image.open("red-triangle.webp")

# Preprocess image
inputs = processor(images=image, return_tensors="pt")

# Forward pass
with torch.no_grad():
    logits = model(**inputs).logits

# Get prediction
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])


Predicted class: envelope


In [ ]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch
import torch.nn.functional as F
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Load processor and model
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = AutoModel.from_pretrained("microsoft/resnet-50")

def get_signature_vector(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        features = model(**inputs).last_hidden_state
    # Pool to a fixed-size vector (e.g., mean pooling)
    vector = features.mean(dim=1)

    # Suppose vectors is a list of signature vectors
    tsne = TSNE(n_components=2)
    reduced = tsne.fit_transform(vectors)

    plt.scatter(reduced[:,0], reduced[:,1], c=labels)  # Labels: same/different person
    plt.title("Signature vector space")
    plt.show()

    
    return vector.squeeze()  # shape: (2048,)

# Load two signatures
vec1 = get_signature_vector("signature1.jpg")
vec2 = get_signature_vector("signature2.jpg")

# Cosine similarity
similarity = F.cosine_similarity(vec1.unsqueeze(0), vec2.unsqueeze(0)).item()
print("Cosine Similarity:", similarity)

# Simple threshold
if similarity > 0.9:
    print("Likely same person")
else:
    print("Likely different person")
